In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
%load_ext tensorboard

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [4]:
from PIL import Image
from IPython.display import display

import os
import numpy as np
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from get_frames import get_frames
from build_model_ed import build_model_1, build_model_2
from build_model_lstm import build_model_lstm_1, build_model_lstm_2, build_model_lstm_3
from create_encoder_decoder_data import process_image, process_dataset, get_full_path
from get_encoder_decoder import get_encoder_decoder

In [5]:
#gpu check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
#load data
unit_numb = 5
test_size = 0.1
stride_step = 50
norm_factor=1
dtype = 'uint8'
file_name = f'save_data//encoded_model_2//encoded_{unit_numb}_{stride_step}_ts_{test_size}_{dtype}_norm_{norm_factor}.pkl'
with open(file_name, 'rb') as f:
    frames_data_encoded = pickle.load(f)
    
print(f'Data shape: {frames_data_encoded.shape}')

Data shape: (770, 80, 5)


In [7]:
#shuffle
np.random.shuffle(frames_data_encoded)

In [8]:
#processing the whole dataset
window_size = 6
window_size_predicted = 1

#test_array = np.copy(frames_data_encoded[:2])

#input data
x_train_lstm = np.lib.stride_tricks.sliding_window_view(
    frames_data_encoded,
    window_shape = window_size,
    axis=1,
)
#cut off last timesteps
x_train_lstm = x_train_lstm[:,:-window_size_predicted,:,:]
#transpose
x_train_lstm = np.transpose(x_train_lstm, axes=(0,1,3,2))
shape = x_train_lstm.shape
x_train_lstm = np.reshape(x_train_lstm,(shape[0]*shape[1], shape[2], shape[3]))

#labels
y_train_lstm = np.lib.stride_tricks.sliding_window_view(
    frames_data_encoded,
    window_shape = window_size_predicted,
    axis=1,
)
#shift
y_train_lstm = y_train_lstm[:,window_size:,:,:]
#transpose
y_train_lstm = np.transpose(y_train_lstm, axes=(0,1,3,2))
#join batches from different timeseries together
shape = y_train_lstm.shape
print(f'Shape before joining: {shape}')
y_train_lstm = np.reshape(y_train_lstm,(shape[0]*shape[1], shape[2], shape[3]))

if window_size_predicted==1:
    #if only one frame is predicted
    #get rid of timestep dim
    shape = y_train_lstm.shape
    y_train_lstm = np.reshape(y_train_lstm,(shape[0], shape[1]*shape[2]))

print(f'Initial data shape: {frames_data_encoded.shape}')
print(f'Input data shape: {x_train_lstm.shape}')
print(f'Output data shape: {y_train_lstm.shape}')

Shape before joining: (770, 74, 1, 5)
Initial data shape: (770, 80, 5)
Input data shape: (56980, 6, 5)
Output data shape: (56980, 5)


In [9]:
cells_list = [20,20]
ifDense = True
input_shape = (None, unit_numb)
loss = 'mean_squared_error'
learning_rate = 0.1
optimizer_name = "Adadelta"
optimizer = tf.keras.optimizers.legacy.Adadelta(learning_rate=learning_rate, name=optimizer_name)
#optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

model = build_model_lstm_3(input_shape, cells_list, ifDense=ifDense, ifDropout=False)
model.compile(loss=loss, 
              optimizer=optimizer, )
              #metrics=metrics)
model.summary() 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 5)]         0         
                                                                 
 lstm (LSTM)                 (None, None, 20)          2080      
                                                                 
 lstm_1 (LSTM)               (None, 20)                3280      
                                                                 
 dense (Dense)               (None, 5)                 105       
                                                                 
Total params: 5465 (21.35 KB)
Trainable params: 5465 (21.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
#check activation function of the lstm layer
model.layers[1].activation

<function keras.src.activations.tanh(x)>

In [10]:
#model training 16 ifDense False bp 

folder = 'lstm_window'
epochs = 2000
batch_size = 74
validation_split = 0.1

cells_list_str = '_'.join(str(x) for x in cells_list)

model_name = f'model_lstm_window_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'
model_name = f'model_lstm_window_units_{unit_numb}_bs74_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'


#define callbacks
# Write TensorBoard logs
log_dir = f'./tensorboard/{folder}/logs/{model_name}'
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
#Stop training when no improvement
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                  patience=10, 
                                                  restore_best_weights=True)
#Reduce learning rate when a metric has stopped improving
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=5)
callbacks = [tensorboard, early_stopping, reduce_lr]

In [51]:
# Clear any logs from previous runs
if 1:
    ld = log_dir
else:
    ld=None
!rm -rf $ld

In [11]:
#units 15
#bs 10 [30, 30]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0285 - val_loss: 0.0236 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 0.0178 - val_loss: 0.0159 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0127 - val_loss: 0.0117 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0093 - val_loss: 0.0089 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0073 - val_loss: 0.0072 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0060 - val_loss: 0.0060 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0051 - val_loss: 0.0053 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0047 - val_loss: 0.0049 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

Epoch 68/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 8.4364e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-05
Epoch 69/2000
5129/5129 [==============================] - 13s 3ms/step - loss: 8.4364e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-05
Epoch 70/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 8.4364e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-05
Epoch 71/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 8.4363e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-06
Epoch 72/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 8.4364e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-06
Epoch 73/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 8.4364e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-06
Epoch 74/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 8.4363e-04 - val_loss: 8.7927e-04 - lr: 1.0000e-06
Epoch 75/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 8.4

In [11]:
#units 10
#bs 10 [20, 20]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 14s 2ms/step - loss: 0.0310 - val_loss: 0.0183 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0151 - val_loss: 0.0119 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0100 - val_loss: 0.0082 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0078 - val_loss: 0.0070 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0068 - val_loss: 0.0061 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0060 - val_loss: 0.0054 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0054 - val_loss: 0.0049 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0049 - val_loss: 0.0045 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

Epoch 68/2000
5129/5129 [==============================] - 13s 3ms/step - loss: 7.3427e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-04
Epoch 69/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3425e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-05
Epoch 70/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3425e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-05
Epoch 71/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3425e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-05
Epoch 72/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3425e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-05
Epoch 73/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3425e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-05
Epoch 74/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3425e-04 - val_loss: 6.9602e-04 - lr: 1.0000e-06
Epoch 75/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 7.3

In [11]:
#units 20
#bs 10 [50, 50]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 19s 3ms/step - loss: 0.0292 - val_loss: 0.0198 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0176 - val_loss: 0.0140 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0126 - val_loss: 0.0104 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0096 - val_loss: 0.0080 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0077 - val_loss: 0.0066 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0064 - val_loss: 0.0056 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0056 - val_loss: 0.0051 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0051 - val_loss: 0.0048 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

In [11]:
#units 20
#bs 10 [40, 40]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 15s 2ms/step - loss: 0.0310 - val_loss: 0.0255 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0190 - val_loss: 0.0152 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0129 - val_loss: 0.0109 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 13s 2ms/step - loss: 0.0096 - val_loss: 0.0085 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 13s 3ms/step - loss: 0.0076 - val_loss: 0.0070 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 0.0064 - val_loss: 0.0062 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 0.0057 - val_loss: 0.0057 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0053 - val_loss: 0.0053 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

In [ ]:
#bs 10 [10, 10, 10, 10]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 24s 4ms/step - loss: 0.0377 - val_loss: 0.0368 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 20s 4ms/step - loss: 0.0295 - val_loss: 0.0280 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 21s 4ms/step - loss: 0.0270 - val_loss: 0.0276 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 20s 4ms/step - loss: 0.0267 - val_loss: 0.0274 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 19s 4ms/step - loss: 0.0263 - val_loss: 0.0266 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 19s 4ms/step - loss: 0.0234 - val_loss: 0.0216 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 19s 4ms/step - loss: 0.0207 - val_loss: 0.0207 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 19s 4ms/step - loss: 0.0199 - val_loss: 0.0196 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

In [11]:
#bs 10 [15,15, 15]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 19s 3ms/step - loss: 0.0357 - val_loss: 0.0301 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0267 - val_loss: 0.0242 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0146 - val_loss: 0.0093 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0071 - val_loss: 0.0066 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 18s 3ms/step - loss: 0.0064 - val_loss: 0.0063 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0062 - val_loss: 0.0060 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0059 - val_loss: 0.0057 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0055 - val_loss: 0.0052 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

In [11]:
#bs 10 [15,15]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0308 - val_loss: 0.0187 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 0.0106 - val_loss: 0.0053 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 13s 3ms/step - loss: 0.0047 - val_loss: 0.0037 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 13s 3ms/step - loss: 0.0037 - val_loss: 0.0033 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 13s 2ms/step - loss: 0.0035 - val_loss: 0.0032 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0033 - val_loss: 0.0030 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 13s 3ms/step - loss: 0.0032 - val_loss: 0.0029 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 14s 3ms/step - loss: 0.0030 - val_loss: 0.0028 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

Epoch 67/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4817e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-06
Epoch 68/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4817e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-06
Epoch 69/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4818e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-06
Epoch 70/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4817e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-07
Epoch 71/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4817e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-07
Epoch 72/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4817e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-07
Epoch 73/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4817e-04 - val_loss: 5.9515e-04 - lr: 1.0000e-07
Epoch 74/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.4

In [11]:
#bs 74 [10,10]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
693/693 [==============================] - 4s 3ms/step - loss: 0.0393 - val_loss: 0.0341 - lr: 0.1000
Epoch 2/2000
693/693 [==============================] - 2s 2ms/step - loss: 0.0365 - val_loss: 0.0323 - lr: 0.1000
Epoch 3/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0339 - val_loss: 0.0296 - lr: 0.1000
Epoch 4/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0301 - val_loss: 0.0257 - lr: 0.1000
Epoch 5/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0254 - val_loss: 0.0216 - lr: 0.1000
Epoch 6/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0206 - val_loss: 0.0169 - lr: 0.1000
Epoch 7/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0157 - val_loss: 0.0125 - lr: 0.1000
Epoch 8/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0119 - val_loss: 0.0098 - lr: 0.1000
Epoch 9/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.00

Epoch 72/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0024 - val_loss: 0.0025 - lr: 0.1000
Epoch 73/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0024 - val_loss: 0.0025 - lr: 0.1000
Epoch 74/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0023 - val_loss: 0.0025 - lr: 0.1000
Epoch 75/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0023 - val_loss: 0.0025 - lr: 0.1000
Epoch 76/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0023 - val_loss: 0.0024 - lr: 0.1000
Epoch 77/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0023 - val_loss: 0.0024 - lr: 0.1000
Epoch 78/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0023 - val_loss: 0.0024 - lr: 0.1000
Epoch 79/2000
693/693 [==============================] - 1s 2ms/step - loss: 0.0022 - val_loss: 0.0024 - lr: 0.1000
Epoch 80/2000
693/693 [==============================] - 1s 2ms/step - l

In [11]:
#bs 10 [10,10]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

Epoch 1/2000
5129/5129 [==============================] - 14s 2ms/step - loss: 0.0312 - val_loss: 0.0201 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0143 - val_loss: 0.0070 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0067 - val_loss: 0.0058 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0058 - val_loss: 0.0054 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 13s 2ms/step - loss: 0.0052 - val_loss: 0.0047 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0041 - val_loss: 0.0036 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0033 - val_loss: 0.0033 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 0.0031 - val_loss: 0.0032 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

Epoch 67/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-07
Epoch 68/2000
5129/5129 [==============================] - 13s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-07
Epoch 69/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-07
Epoch 70/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-07
Epoch 71/2000
5129/5129 [==============================] - 11s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-08
Epoch 72/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-08
Epoch 73/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.5319e-04 - val_loss: 7.2482e-04 - lr: 1.0000e-08
Epoch 74/2000
5129/5129 [==============================] - 12s 2ms/step - loss: 6.5

In [10]:
#bs 10 [10,20,30]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

#model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

Epoch 1/2000
5129/5129 [==============================] - 19s 3ms/step - loss: 0.0358 - val_loss: 0.0295 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0252 - val_loss: 0.0216 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0152 - val_loss: 0.0109 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0093 - val_loss: 0.0060 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0063 - val_loss: 0.0056 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0058 - val_loss: 0.0050 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 0.0049 - val_loss: 0.0038 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 0.0039 - val_loss: 0.0035 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

Epoch 68/2000
5129/5129 [==============================] - 18s 3ms/step - loss: 6.7033e-04 - val_loss: 7.1119e-04 - lr: 1.0000e-04
Epoch 69/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 6.7032e-04 - val_loss: 7.1119e-04 - lr: 1.0000e-04
Epoch 70/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 6.7031e-04 - val_loss: 7.1118e-04 - lr: 1.0000e-04
Epoch 71/2000
5129/5129 [==============================] - 18s 3ms/step - loss: 6.7030e-04 - val_loss: 7.1120e-04 - lr: 1.0000e-04
Epoch 72/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 6.7029e-04 - val_loss: 7.1120e-04 - lr: 1.0000e-05
Epoch 73/2000
5129/5129 [==============================] - 18s 3ms/step - loss: 6.7029e-04 - val_loss: 7.1120e-04 - lr: 1.0000e-05
Epoch 74/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 6.7029e-04 - val_loss: 7.1119e-04 - lr: 1.0000e-05
Epoch 75/2000
5129/5129 [==============================] - 18s 3ms/step - loss: 6.7

In [15]:
#bs 10 [10,10,10]
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

#model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

Epoch 1/2000
5129/5129 [==============================] - 19s 3ms/step - loss: 0.0373 - val_loss: 0.0360 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0267 - val_loss: 0.0213 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0143 - val_loss: 0.0123 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0103 - val_loss: 0.0088 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0070 - val_loss: 0.0062 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0060 - val_loss: 0.0051 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0046 - val_loss: 0.0037 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 0.0040 - val_loss: 0.0035 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] 

Epoch 67/2000
5129/5129 [==============================] - 19s 4ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-06
Epoch 68/2000
5129/5129 [==============================] - 15s 3ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-06
Epoch 69/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-06
Epoch 70/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-06
Epoch 71/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-06
Epoch 72/2000
5129/5129 [==============================] - 18s 4ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-07
Epoch 73/2000
5129/5129 [==============================] - 17s 3ms/step - loss: 6.1523e-04 - val_loss: 5.1632e-04 - lr: 1.0000e-07
Epoch 74/2000
5129/5129 [==============================] - 16s 3ms/step - loss: 6.1

In [21]:
#bs 10 single cell
model.fit(
    x_train_lstm,
    y_train_lstm,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks,
)

#model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

Epoch 1/2000
5129/5129 [==============================] - 10s 2ms/step - loss: 0.0286 - val_loss: 0.0180 - lr: 0.1000
Epoch 2/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0100 - val_loss: 0.0048 - lr: 0.1000
Epoch 3/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0038 - val_loss: 0.0031 - lr: 0.1000
Epoch 4/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0030 - val_loss: 0.0027 - lr: 0.1000
Epoch 5/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0027 - val_loss: 0.0024 - lr: 0.1000
Epoch 6/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0024 - val_loss: 0.0022 - lr: 0.1000
Epoch 7/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0022 - val_loss: 0.0020 - lr: 0.1000
Epoch 8/2000
5129/5129 [==============================] - 8s 2ms/step - loss: 0.0021 - val_loss: 0.0019 - lr: 0.1000
Epoch 9/2000
5129/5129 [==============================] - 8s 2m

In [12]:
model.save_weights(f'saved_models/{folder}/{model_name}/{model_name}')

In [11]:
#tensorboard results
folder = 'lstm_window'
cells_list = [10,10,10]
cells_list_str = '_'.join(str(x) for x in cells_list)


model_name = f'model_lstm_window_units_{unit_numb}_bs2_{input_shape[-1]}_{cells_list_str}_ifDense_{ifDense}'

log_dir = f'./tensorboard/{folder}/logs/{model_name}'
%tensorboard --logdir $log_dir